## Setup Software and Some Libraries

We first need to set the working environment and the path to the dataset. The SPINE framework is imported from the local installation, and we configure the data loader to read the reconstructed HDF5 files.

### Environment Configuration
- **Software directory**: Contains SPINE v0.2.2 installation
- **Data directory**: Contains the MiniRun6.1 reconstructed files

### SPINE Driver Configuration
The driver is configured to:
- Read HDF5 files with reconstruction output
- Build both true and reconstructed representations
- Use cm units for spatial coordinates
- Reconstruct particles and interactions (not individual fragments)

In [1]:
import sys

SOFTWARE_DIR = "/home/azam/spine_bilal/spine_v0.2.2"
DATA_DIR = "/home/azam/comprehensive_exam/spine_files/MR6.1/"
sys.path.append(SOFTWARE_DIR)

import numpy as np
import math
import pandas as pd
from collections import OrderedDict
from scipy.spatial.distance import cdist

In [2]:
import yaml
from spine.driver import Driver

DATA_PATH = DATA_DIR + 'MiniRun6.1_1E19_RHC.spine.0000492.MLRECO_SPINE.hdf5'

cfg = '''
# Load HDF5 files
io:
    reader:
        name: hdf5
        file_keys: DATA_PATH
        skip_unknown_attrs: true
# Build reconstruction output representations
build:
    mode: both
    units: cm
    fragments: false
    particles: true
    interactions: true
'''.replace('DATA_PATH', DATA_PATH)

cfg = yaml.safe_load(cfg)
driver = Driver(cfg)

Welcome to JupyROOT 6.30/06

 ██████████   ██████████    ███   ███       ██   ███████████
███        █  ██       ███   █    █████     ██   ██         
  ████████    ██       ███  ███   ██  ████  ██   ██████████ 
█        ███  ██████████     █    ██     █████   ██         
 ██████████   ██            ███   ██       ███   ███████████

Release version: 0.2.2

$CUDA_VISIBLE_DEVICES=

Configuration processed at: Linux x3004c0s25b0n0 6.4.0-150600.23.53-default #1 SMP PREEMPT_DYNAMIC Wed Jun  4 05:37:40 UTC 2025 (2d991ff) x86_64 x86_64 x86_64 GNU/Linux

base: {seed: 1762372557}
io:
  reader: {name: hdf5, file_keys: /home/azam/comprehensive_exam/spine_files/MR6.1/MiniRun6.1_1E19_RHC.spine.0000492.MLRECO_SPINE.hdf5,
    skip_unknown_attrs: true}
build: {mode: both, units: cm, fragments: false, particles: true, interactions: true}

Will load 1 file(s):
  - /home/azam/comprehensive_exam/spine_files/MR6.1/MiniRun6.1_1E19_RHC.spine.0000492.MLRECO_SPINE.hdf5

Total number of entries in the file(s): 

## Interaction Details Printer

This cell prints detailed information for a selected **reconstructed interaction** and its **matched truth interaction(s)** using the SPINE driver output.

### Features
- Shows **primary charged track multiplicity** (μ, π, p, K only) for both reco and truth.
- Lists per-particle kinematics: kinetic energy, momentum, length, and primary flag.
- Displays matching information, overlap scores, and interaction properties.

### Example Usage
```python
# Display a specific truth interaction
print_interaction_details(entry_number, reco_interaction_id, truth_interaction_id)


In [3]:
def print_interaction_details(entry_number, reco_interaction_id, truth_interaction_id=None):
    """
    Print detailed information about a specific reco interaction and its matched truth interaction.
    Shows PRIMARY CHARGED TRACK multiplicity (muons, pions, protons, kaons only).
    
    Parameters:
    -----------
    entry_number : int
        Entry number in the dataset
    reco_interaction_id : int
        Reco interaction ID to inspect
    truth_interaction_id : int, optional
        Specific truth interaction ID to display. If None, displays all matched truth interactions.
    """
    
    data = driver.process(entry=entry_number)
    
    # Find the reco interaction
    reco_ixn = None
    for ixn in data['reco_interactions']:
        if ixn.id == reco_interaction_id:
            reco_ixn = ixn
            break
    
    if reco_ixn is None:
        print(f"❌ Reco interaction {reco_interaction_id} not found in entry {entry_number}")
        return
    
    # Get reco particles for this interaction
    reco_particles = [p for p in data['reco_particles'] if p.interaction_id == reco_interaction_id]
    
    print("=" * 80)
    print(f"ENTRY {entry_number} | RECO INTERACTION {reco_interaction_id}")
    print("=" * 80)
    
    # RECO section
    print("\nRECONSTRUCTED (RECO)")
    print("-" * 80)
    print(f"Total reco particles: {len(reco_particles)}")
    print(f"Reco vertex: {reco_ixn.vertex}")
    print(f"Is contained: {reco_ixn.is_contained}")
    print(f"Is fiducial: {reco_ixn.is_fiducial}")
    
    # Reco multiplicity - PRIMARY CHARGED TRACKS ONLY
    print(f"\nReco Multiplicity (Primary Charged Tracks Only):")
    reco_primary_charged = [p for p in reco_particles if p.is_primary and p.pid in [2, 3, 4, 5]]
    reco_primary_pids = [p.pid for p in reco_primary_charged]
    
    n_mu_reco = reco_primary_pids.count(2)
    n_pion_reco = reco_primary_pids.count(3)
    n_proton_reco = reco_primary_pids.count(4)
    n_kaon_reco = reco_primary_pids.count(5)
    
    print(f"  Muons (pid=2):     {n_mu_reco}")
    print(f"  Pions (pid=3):     {n_pion_reco}")
    print(f"  Protons (pid=4):   {n_proton_reco}")
    print(f"  Kaons (pid=5):     {n_kaon_reco}")
    print(f"  Total primary charged: {len(reco_primary_charged)}")
    
    # Reco particle breakdown
    print(f"\nReco Particle Breakdown:")
    for i, p in enumerate(reco_particles):
        pid_name = {2: "Muon", 3: "Pion", 4: "Proton", 5: "Kaon", 1: "Electron", 0: "Photon"}.get(p.pid, "Unknown")
        print(f"  [{i}] PID={p.pid} ({pid_name}): KE={p.ke:.3f} MeV, p={p.p:.3f} MeV/c, length={p.length:.3f} cm, primary={p.is_primary}")
    
    # TRUTH section
    print("\n" + "=" * 80)
    print("TRUTH INFORMATION")
    print("-" * 80)
    
    if reco_ixn.is_matched:
        matched_truth_ids = reco_ixn.match_ids
        overlap_scores = reco_ixn.match_overlaps
        
        print(f"✅ Is matched: YES")
        print(f"Matched to truth interaction ID(s): {matched_truth_ids}")
        print(f"Overlap score(s): {overlap_scores}")
        
        # Determine which truth interactions to display
        if truth_interaction_id is not None:
            # Display only the specified truth interaction
            if truth_interaction_id not in matched_truth_ids:
                print(f"\n⚠️  Warning: Truth interaction {truth_interaction_id} is not matched to reco interaction {reco_interaction_id}")
                print(f"    Matched truth interactions are: {matched_truth_ids}")
                print("    Proceeding to display requested truth interaction anyway...\n")
            
            truth_ixns = [t for t in data['truth_interactions'] if t.id == truth_interaction_id]
        else:
            # Display all matched truth interactions
            truth_ixns = [t for t in data['truth_interactions'] if t.id in matched_truth_ids]
        
        for truth_ixn in truth_ixns:
            print(f"\nTruth Interaction {truth_ixn.id}:")
            truth_particles = [p for p in data['truth_particles'] if p.interaction_id == truth_ixn.id]
            
            print(f"  Total truth particles: {len(truth_particles)}")
            print(f"  Truth vertex: {truth_ixn.vertex}")
            print(f"  Interaction type: {truth_ixn.interaction_type}")
            print(f"  Current type: {truth_ixn.current_type}")
            
            # Truth multiplicity - PRIMARY CHARGED TRACKS ONLY
            print(f"\n  Truth Multiplicity (Primary Charged Tracks Only):")
            truth_primary_charged = [p for p in truth_particles if p.is_primary and p.pid in [2, 3, 4, 5]]
            truth_primary_pids = [p.pid for p in truth_primary_charged]
            
            n_mu_truth = truth_primary_pids.count(2)
            n_pion_truth = truth_primary_pids.count(3)
            n_proton_truth = truth_primary_pids.count(4)
            n_kaon_truth = truth_primary_pids.count(5)
            
            print(f"    Muons (pid=2):     {n_mu_truth}")
            print(f"    Pions (pid=3):     {n_pion_truth}")
            print(f"    Protons (pid=4):   {n_proton_truth}")
            print(f"    Kaons (pid=5):     {n_kaon_truth}")
            print(f"    Total primary charged: {len(truth_primary_charged)}")
            
            # Truth particle breakdown
            print(f"\n  Truth Particle Breakdown:")
            for i, p in enumerate(truth_particles):
                pid_name = {2: "Muon", 3: "Pion", 4: "Proton", 5: "Kaon", 1: "Electron", 0: "Photon"}.get(p.pid, "Unknown")
                print(f"    [{i}] PID={p.pid} ({pid_name}): KE={p.ke:.3f} MeV, p={p.p:.3f} MeV/c, length={p.length:.3f} cm, primary={p.is_primary}, pdg={p.pdg_code}")
    else:
        print(f"❌ Is matched: NO - No truth interaction to display")
    
    print("\n" + "=" * 80)

    # --- Reco–Truth Particle Matching Table ---
    if reco_ixn.is_matched:
        print("\n Reco–Truth Particle Matching Table")
        print("  " + "-" * 75)
        print(f"  {'Reco Index':<12} {'Reco PID':<10} {'Reco Type':<12}"
              f"{'Truth Index':<12} {'Truth PID':<10} {'Truth Type':<12}")
        print("  " + "-" * 75)
    
        pid_map = {2: "Muon", 3: "Pion", 4: "Proton", 5: "Kaon", 1: "Electron", 0: "Photon"}
        for i, reco_p in enumerate(reco_particles):
            if not hasattr(reco_p, "match_ids") or len(reco_p.match_ids) == 0:
                continue
    
            truth_match_id = reco_p.match_ids[0]
            truth_p_matches = [tp for tp in truth_particles if tp.id == truth_match_id]
            if len(truth_p_matches) == 0:
                continue
    
            truth_p = truth_p_matches[0]
            dp = abs(reco_p.p - truth_p.p)
    
            print(f"  {i:<12} {reco_p.pid:<10} {pid_map.get(reco_p.pid, 'Unknown'):<12}"
                  f"{truth_p.id:<12} {truth_p.pid:<10} {pid_map.get(truth_p.pid, 'Unknown'):<12}")
    else:
        print("\n  ⚠️  No reco–truth particle matching information available.")

entry_number = 28
reco_interaction_id = 0
truth_interaction_id = 0

# Example usage:
# Display all matched truth interactions (original behavior)
# print_interaction_details(entry_number, reco_interaction_id)

# Display only a specific truth interaction
print_interaction_details(entry_number, reco_interaction_id, truth_interaction_id)

ENTRY 28 | RECO INTERACTION 0

RECONSTRUCTED (RECO)
--------------------------------------------------------------------------------
Total reco particles: 10
Reco vertex: [-47.6655 -33.9201 -42.7881]
Is contained: False
Is fiducial: True

Reco Multiplicity (Primary Charged Tracks Only):
  Muons (pid=2):     1
  Pions (pid=3):     0
  Protons (pid=4):   5
  Kaons (pid=5):     0
  Total primary charged: 6

Reco Particle Breakdown:
  [0] PID=1 (Electron): KE=14.678 MeV, p=15.181 MeV/c, length=-1.000 cm, primary=False
  [1] PID=4 (Proton): KE=110.479 MeV, p=468.533 MeV/c, length=9.312 cm, primary=True
  [2] PID=4 (Proton): KE=353.267 MeV, p=887.535 MeV/c, length=66.784 cm, primary=True
  [3] PID=4 (Proton): KE=408.059 MeV, p=965.532 MeV/c, length=84.141 cm, primary=True
  [4] PID=2 (Muon): KE=287.302 MeV, p=378.489 MeV/c, length=116.141 cm, primary=True
  [5] PID=4 (Proton): KE=126.952 MeV, p=504.329 MeV/c, length=11.879 cm, primary=True
  [6] PID=4 (Proton): KE=76.819 MeV, p=387.371 MeV/c

## Event Visualization with Camera Synchronization

This cell visualizes **reconstructed vs. truth-level particles** for a selected interaction using the `spine.vis.out.Drawer` interface.  
It automatically generates an interactive **3D Plotly HTML visualization** with synchronized camera motion between the reco and truth scenes.

### Features
- Displays **Reconstructed** and **Truth** particles side-by-side (`split_scene=True`).
- Uses consistent **camera synchronization**, allowing both scenes to rotate and zoom together.

In [4]:
from spine.vis.out import Drawer
import plotly.io as pio
import os
from IPython.display import IFrame, display, HTML

def add_camera_sync_to_html(html_file_path):
    js_sync_code = '''
        <script type="text/javascript">
          const graph = document.querySelector('.plotly-graph-div');
          const scenes = ['scene', 'scene2'];
          let isSyncing = false;
          graph.on('plotly_relayout', (eventdata) => {
            if (isSyncing) return;
            let camScene = null;
            let camData = null;
            for (const sceneName of scenes) {
              const key = sceneName + '.camera';
              if (eventdata.hasOwnProperty(key)) {
                camScene = sceneName;
                camData = eventdata[key];
                break;
              }
            }
            if (!camScene) return; // No camera update found
            isSyncing = true;
            // Update all other scenes with the same camera
            scenes.forEach(sceneName => {
              if (sceneName !== camScene) {
                let update = {};
                update[sceneName + '.camera'] = camData;
                Plotly.relayout(graph, update).catch(() => {}).then(() => {
                  isSyncing = false;
                });
              }
            });
          });
          console.log("Camera synchronization enabled safely for scenes in one plot.");
        </script>
        '''
    with open(html_file_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    new_html_content = html_content.replace('</body>', js_sync_code + '\n</body>')
    with open(html_file_path, 'w', encoding='utf-8') as f:
        f.write(new_html_content)
    print(f"Inserted camera sync JavaScript into {html_file_path}")

def visualize_and_sync(entry_number, reco_interaction_id, truth_interaction_id=None):
    data = driver.process(entry=entry_number)
    file_number = os.path.basename(DATA_PATH).split('.')[3]
    
    if truth_interaction_id is None:
        truth_interaction_id = reco_interaction_id
    
    reco_particles = [p for p in data['reco_particles'] if p.interaction_id == reco_interaction_id]
    truth_particles = [p for p in data['truth_particles'] if p.interaction_id == truth_interaction_id]
    reco_interactions = [ixn for ixn in data['reco_interactions'] if ixn.id == reco_interaction_id]
    truth_interactions = [ixn for ixn in data['truth_interactions'] if ixn.id == truth_interaction_id]
    
    data['reco_particles'] = reco_particles
    data['truth_particles'] = truth_particles
    data['reco_interactions'] = reco_interactions
    data['truth_interactions'] = truth_interactions
    
    drawer = Drawer(data, draw_mode='both', detector='2x2', split_scene=True)
    fig = drawer.get(
        obj_type='particles',
        attr='pid',
        draw_end_points=True,
        draw_vertices=True,
        synchronize=False,
        titles=['Reconstructed Particles', 'Truth Particles'],
        split_traces=True
    )
    fig.update_layout(height=800, width=1000)
    
    html_filename = f"MR6_{file_number}_entry_{entry_number}_reco_{reco_interaction_id}_truth_{truth_interaction_id}.html"
    pio.write_html(fig, html_filename, full_html=True)
    add_camera_sync_to_html(html_filename)
    print(f"Saved and synchronized HTML: {html_filename}")
    
    # Display the HTML file in Jupyter using IFrame
    display(IFrame(src=html_filename, width=1000, height=800))

# Example usage
visualize_and_sync(entry_number, reco_interaction_id, truth_interaction_id)

Inserted camera sync JavaScript into MR6_0000492_entry_28_reco_0_truth_0.html
Saved and synchronized HTML: MR6_0000492_entry_28_reco_0_truth_0.html
